In [27]:
dayObs = 'Today'
run = "None"
sensor='R10_S10'
guidersensor='R44_SG0'

In [36]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta
import tabulate
import traceback

In [29]:
if dayObs.lower()=='today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')

In [30]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
#    'LSSTCam/calib/unbounded',
#    'LSSTCam/photodiode'
])

In [31]:
exposureList = []

if run == "None":
    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"
else:
    where=f"exposure.science_program='{run}' and instrument='LSSTCam'"    

for record in butler.registry.queryDimensionRecords("exposure", where=where):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    guider=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, guider])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            f'<a href=http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id} target="_blank">{record.obs_id}</a>',
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            f'<a href=https://usdf-rsp.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/GuiderData?expid={record.obs_id}&sensor={guidersensor}&ts_hide_code=1 target="_blank">Guider</a>',
            f'<a href=https://usdf-rsp.slac.stanford.edu/times-square/github/lsst/CameraTimesSquare/Run7/FitsHeader?expid={record.obs_id}&detector={sensor} target="_blank" >meta</a>',    
                     )
            for [id,record,nimage, guider] in exposureList ]
            )

In [32]:
try:
    subtable=Table(rows=[  ( record.obs_id, record.seq_num, nimage ) for [id,record,nimage, guider] in exposureList ], names=(  'id', 'seqnum', 'nimage'  ) )
except:
    traceback.print_exc()
    subtable=None   

# Possible data transfer issue

In [ ]:
if subtable is not None:
    for a, b in zip(subtable[:-1][numpy.diff(subtable['seqnum'])!=1], subtable[1:][numpy.diff(subtable['seqnum'])!=1]):
        print(a['id'], b['id'])

# Changes in number of images ingested

In [34]:
for a, b in zip(subtable[:-1][numpy.diff(subtable['nimage'])!=0], subtable[1:][numpy.diff(subtable['nimage'])!=0]):
    print(a['id'], a['nimage'], b['id'], b['nimage'])

# Full records

In [35]:
print(f"The number of exposures is {len(table)}")
if len(table)>0:
    tabulate.tabulate(table[::-1], tablefmt='unsafehtml',headers=[
                    'obs_id', 'time', 'seqnum', 'observation_type',
                    'science_program', 'observation_reason','exposure_time',
                    'physical_filter', 'nimage', 'Guider', 'metadata'
                    ])

The number of exposures is 0
